In [1]:
import pandas as pd

# 1 -  Fuzzy Matching

Utilizaremos los conceptos de fuzzy matching para mejorar una variable categórica

In [ ]:
#Primero instalemos fuzzywuzzy
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Leemos un archivo con información de los departamentos de Uruguay
fuzzy = pd.read_csv("/content/drive/MyDrive/Curso CPE Data Wrangling/Clases/Clase 10 - Manejo avanzado strings e IO datos/Ejercicio_Fuzzy.csv", encoding = "ISO-8859-1")

In [ ]:
fuzzy.sample(6)

,Id,Departamento,Valor
12,13,durazno,27
6,7,C. Largo,29
76,77,Flors,41
77,78,Flores,32
10,11,Montev.,89
61,62,Tacuarmebo,79


Tenemos un dataset de ejemplo con un campo "Departamento" ingresado de forma manual, con errores de tipeo

In [ ]:
fuzzy['Departamento'].drop_duplicates()

0         Montevideo
1          Canelones
2           San Jose
3            Soriano
4            Durazno
5        Cerro Largo
6           C. Largo
7            S. Jos
8           San Jos
9             Mvdeo.
10           Montev.
11           Colonia
12           durazno
14    Treinta y Tres
15      Treintaytres
16         Rio Negro
18          R. negro
19         Maldonado
21    Trainta y trs
22         maldonado
23             Rocha
24        Tacuarmebo
25       Tamcuaremb¢
26          Canelons
27           Artigas
28             Salto
29            Rivera
32          Paysand£
33          Paysandu
75           Florida
76             Flors
77            Flores
79         Lavalleja
Name: Departamento, dtype: object

De la librería fuzzywuzzy usaremos las funciones:


`ratio(string_1, string_2)`: calcula el ratio de coincidencia entre string_1 y string_2 usando la distancia de Levenshtein

`partial_ratio(string_1, string_2)`: calcula el ratio de coincidencia tomando al string más corto como substring y buscando coincidencias parciales



In [ ]:
# Ratio entre strings iguales
fuzz.ratio('durazno','durazno')

100

In [ ]:
# Ratio entre strings levemente distintos
fuzz.ratio('Durazno','durazno')

86

In [ ]:
# Ratio entre strings muy distintos
fuzz.ratio('durazno','Yo vivo en durazno hace años')

40

In [ ]:
# Ratio parcial entre strings muy distintos
fuzz.partial_ratio('durazno','Yo vivo en durazno hace años')

100

Obtenemos una lista con los valores verdaderos de los departamentos.

In [ ]:
departamentos = [
    'Montevideo',
    'Treinta y Trés',
    'Colonia',
    'Artigas',
    'Canelones',
    'Cerro Largo',
    'Durazno',
    'Flores',
    'Florida',
    'Lavalleja',
    'Maldonado',
    'Paysandú',
    'Río Negro',
    'Rivera',
    'Rocha',
    'Salto',
    'San José',
    'Soriano',
    'Tacuarembó'
]

Busquemos encontrar el valor más próximo para uno de los valores de nuestro dataset.
En general es un buen procedimiento utilizar las técnicas de manipulación de strings como quitar eliminar espacios en blanco, llevar todo el texto a minuscula, etc. previo a usar las funciones de fuzzy matching funciones.

In [ ]:
fuzzy.loc[76,'Departamento']

'Flors'

In [ ]:
depto = fuzzy.loc[76,'Departamento'].lower()
depto

'flors'

In [ ]:
max_coincidencia = 0
# Recorremos la lista de departamentos
for valor_correcto in departamentos:
  # Llevamos a minuscula
  valor_correcto = valor_correcto.lower()
  # Obtenemos el ratio de coincidencia
  coincidencia = fuzz.ratio(depto, valor_correcto)
  # Si el puntaje de la coincidencia es mayor que la vigente
  if coincidencia > max_coincidencia:
    # Guardar el valor correcto
    correccion = valor_correcto
    # Reemplazar el puntaje
    max_coincidencia = coincidencia

print(f"Para el valor {depto}, el valor correcto es {correccion} con un puntaje de {max_coincidencia}")

Para el valor flors, el valor correcto es flores con un puntaje de 91


In [ ]:
for _,row in fuzzy.iterrows():
  depto = row['Departamento'].lower()
  max_coincidencia = 0
  # Recorremos la lista de departamentos
  for valor_correcto in departamentos:
    # Llevamos a minuscula
    valor_correcto = valor_correcto.lower()
    # Obtenemos el ratio de coincidencia
    coincidencia = fuzz.ratio(depto, valor_correcto)
    # Si el puntaje de la coincidencia es mayor que la vigente
    if coincidencia > max_coincidencia:
      # Guardar el valor correcto
      correccion = valor_correcto
      # Reemplazar el puntaje
      max_coincidencia = coincidencia
  if max_coincidencia < 100:
    print(f"Para el valor {depto}, el valor correcto es {correccion} con un puntaje de {max_coincidencia}")

Para el valor san jose, el valor correcto es san josé con un puntaje de 88
Para el valor c. largo, el valor correcto es cerro largo con un puntaje de 74
Para el valor s. jos, el valor correcto es san josé con un puntaje de 67
Para el valor san jos, el valor correcto es san josé con un puntaje de 88
Para el valor mvdeo., el valor correcto es montevideo con un puntaje de 62
Para el valor montev., el valor correcto es montevideo con un puntaje de 71
Para el valor treinta y tres, el valor correcto es treinta y trés con un puntaje de 93
Para el valor treintaytres, el valor correcto es treinta y trés con un puntaje de 85
Para el valor rio negro, el valor correcto es río negro con un puntaje de 89
Para el valor r. negro, el valor correcto es río negro con un puntaje de 82
Para el valor trainta y trs, el valor correcto es treinta y trés con un puntaje de 86
Para el valor tacuarmebo, el valor correcto es tacuarembó con un puntaje de 80
Para el valor tamcuaremb¢, el valor correcto es tacuarem

# 2 - Ejercicios con expresiones regulares

In [40]:
entrevistas_dict = {'nombre':['Pedro', 'Celeste', 'Mariela', 'Jorge', 'Matías'],
 'fecha_nacimiento':['22-03-1994', 'Nací el 15-05-1997', '03-11-91', 'fecha nacimiento: 10-10-1998', '03-01-1999'],
 'titulo_maximo': ['Lic. Administracion', 'Ingeniera en sistemas', 'Licenciada en Economía', 'Ing. industrial', 'Biología'],
 'idiomas':['español', 'español, ingles', 'español', 'español, ingles y portugues', '4']}

In [41]:
entrevistas = pd.DataFrame(entrevistas_dict)

In [42]:
entrevistas

,nombre,fecha_nacimiento,titulo_maximo,idiomas
0,Pedro,22-03-1994,Lic. Administracion,español
1,Celeste,Nací el 15-05-1997,Ingeniera en sistemas,"español, ingles"
2,Mariela,03-11-91,Licenciada en Economía,español
3,Jorge,fecha nacimiento: 10-10-1998,Ing. industrial,"español, ingles y portugues"
4,Matías,03-01-1999,Biología,4


Supongamos que queremos crear una variable limpia con la fecha de nacimiento

In [43]:
# Usamos el método findall 
entrevistas['fecha_nacimiento'].str.findall(pat="\d{2}-\d{2}-\d{2,4}")

0    [22-03-1994]
1    [15-05-1997]
2      [03-11-91]
3    [10-10-1998]
4    [03-01-1999]
Name: fecha_nacimiento, dtype: object

Ahora queremos crear una variable que nos indique si la persona estudió ingeniería 

In [39]:
#Usamos el método contains
entrevistas['titulo_maximo'].str.contains(pat='Ing|ingenier')

0    False
1     True
2    False
3     True
Name: titulo_maximo, dtype: bool

Podemos simplificar aún más la expresión regular

In [38]:
entrevistas['titulo_maximo'].str.contains(pat='I|ing')

0    False
1     True
2    False
3     True
Name: titulo_maximo, dtype: bool

Ahora queremos analizar la variable de idiomas. En primer lugar queremos controlar que las personas hayan puesto un listado de idiomas y no otro dato

In [44]:
entrevistas['idiomas'].str.contains(pat="\d")

0    False
1    False
2    False
3    False
4     True
Name: idiomas, dtype: bool

Vemos que el último registro tiene un dato numérico, el cual no nos sirve para este análisis que queremos realizar. Procedemos a generar una variable con la lista de idiomas

In [45]:
entrevistas['idiomas'].str.split(pat="[,y]")

0                          [español]
1                 [español,  ingles]
2                          [español]
3    [español,  ingles ,  portugues]
4                                [4]
Name: idiomas, dtype: object

Si quisieramos generar una columna por cada elemento podemos utilizar el argumento `expand=True`

In [46]:
entrevistas['idiomas'].str.split(pat="[,y]", expand=True)

,0,1,2
0,español,None,None
1,español,ingles,None
2,español,None,None
3,español,ingles,portugues
4,4,None,None


# 3 - Entrada y salida de datos

Vimos que existe una gran cantidad de formatos de archivos y pandas brinda métodos para leer y escribir en cada uno de ellos.

Uno de los formatos más utilizados es el csv. Observemos algunos de los parámetros más usuales al leer un archivo

In [55]:
# Leemos el archivo de ejemplo que viene en google colab
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv")
housing_df.sample(6)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
5117,-118.12,33.76,45.0,3035.0,516.0,1127.0,527.0,7.0796,500001.0
944,-117.09,32.83,15.0,4138.0,636.0,2001.0,677.0,4.8419,264000.0
4363,-118.02,33.96,36.0,2002.0,361.0,913.0,311.0,4.5446,244700.0
2112,-117.35,34.01,23.0,3707.0,769.0,1938.0,658.0,2.7250,95300.0
6060,-118.22,33.91,28.0,1847.0,500.0,2263.0,473.0,1.5161,103200.0
10288,-120.04,36.93,11.0,3606.0,699.0,2074.0,644.0,2.6941,63300.0


In [56]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           17000 non-null  float64
 1   latitude            17000 non-null  float64
 2   housing_median_age  17000 non-null  float64
 3   total_rooms         17000 non-null  float64
 4   total_bedrooms      17000 non-null  float64
 5   population          17000 non-null  float64
 6   households          17000 non-null  float64
 7   median_income       17000 non-null  float64
 8   median_house_value  17000 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB


In [59]:
# Especificamos una columna de indice
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv", index_col='longitude')
housing_df.head(6)

,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,,,,,,,,
-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
-114.58,33.63,29.0,1387.0,236.0,671.0,239.0,3.3438,74000.0


In [58]:
# Especificamos una fila como nombre de columnas
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv", header=1)
housing_df.sample(6)

,-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000
8085,-118.42,33.90,37.0,1576.0,345.0,662.0,340.0,5.3080,500001.0
8055,-118.42,34.05,33.0,2921.0,652.0,1124.0,608.0,5.0151,500001.0
1844,-117.27,33.18,4.0,3371.0,773.0,1481.0,627.0,2.9133,215700.0
12077,-121.43,38.69,28.0,927.0,165.0,542.0,148.0,2.5000,96200.0
1292,-117.15,33.02,4.0,15029.0,2279.0,5613.0,1696.0,7.2731,450400.0
13421,-121.94,37.26,43.0,2104.0,388.0,1137.0,403.0,4.9236,238000.0


In [60]:
# Indicamos que lea las primeras 1000 filas
housing_df = pd.read_csv("/content/sample_data/california_housing_train.csv", nrows=1000)
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           1000 non-null   float64
 1   latitude            1000 non-null   float64
 2   housing_median_age  1000 non-null   float64
 3   total_rooms         1000 non-null   float64
 4   total_bedrooms      1000 non-null   float64
 5   population          1000 non-null   float64
 6   households          1000 non-null   float64
 7   median_income       1000 non-null   float64
 8   median_house_value  1000 non-null   float64
dtypes: float64(9)
memory usage: 70.4 KB


Hemos viste que podemos leer archivos pasando una URL en la función correspondiente. 
Una función muy útil para leer tablas desde ciertos sitios web es `pandas.read_html(io, match)`:

*   **io**: dirección o nombre del archivo
*   **match**: string o regex para encontrar tablas particulares

La función devuelve una lista de DataFrames con todas las tablas en el html.



In [62]:
? pd.read_html()

Veamos un caso de análisis para la página de Wikipedia de la [Copa Mundial de Fútbol](https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol) 

In [64]:
#Busquemos ahora tablas dentro de una pagina web de wikipedia
mundial = pd.read_html('https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol')
print(f'Total de tablas en la pagina: {len(mundial)}')

Total de tablas en la pagina: 11


In [67]:
# Veamos la quinta tabla: tabla de goleadores del mundial
mundial[4]

,Pos.,Jugador[114]​,Selección,Goles,Partidos,Promedio,Torneos jugados
0,1.0,Miroslav Klose,Alemania,16,24,67,"2002, 2006, 2010 y 2014"
1,2.0,Ronaldo,Brasil,15,19,79,"1998, 2002, 2006"
2,3.0,Gerd Müller,Alemania,14,13,108,1970 y 1974
3,4.0,Just Fontaine,Francia,13,6,217,1958
4,5.0,Pelé,Brasil,12,14,86,"1958, 1962, 1966 y 1970"
5,6.0,Sándor Kocsis,Hungría,11,5,22,1954
6,NaN,Jürgen Klinsmann,Alemania,11,17,65,"1990, 1994 y 1998"
7,8.0,Helmut Rahn,Alemania,10,10,1,1954 y 1958
8,NaN,Gabriel Batistuta,Argentina,10,12,83,"1994, 1998 y 2002"
9,NaN,Gary Lineker,Inglaterra,10,12,83,1986 y 1990


Podemos utilizar el argumento match para obtener un subconjunto de las tablas

In [76]:
#Traigamos la tabla del balon de oro
tabla_balon_oro = pd.read_html('https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol',match='Oro')
print(f'Total de tablas en la pagina: {len(tabla_balon_oro)}')

Total de tablas en la pagina: 1


In [77]:
df = tabla_balon_oro[0]
df

,Copa Mundial,Balón de Oro,Equipo,Balón de Plata,Equipo.1,Balón de Bronce,Equipo.2
0,España 1982,Paolo Rossi,Italia,Falcão,Brasil,Karl-Heinz Rummenigge,Alemania
1,México 1986,Diego Maradona,Argentina,Harald Schumacher,Alemania,Preben Elkjær Larsen,Dinamarca
2,Italia 1990,Salvatore Schillaci,Italia,Lothar Matthäus,Alemania,Diego Maradona,Argentina
3,Estados Unidos 1994,Romário,Brasil,Roberto Baggio,Italia,Hristo Stoitchkov,Bulgaria
4,Francia 1998,Ronaldo,Brasil,Davor Šuker,Croacia,Lilian Thuram,Francia
5,Corea del Sur / Japón 2002,Oliver Kahn,Alemania,Ronaldo,Brasil,Hong Myung-bo,Corea del Sur
6,Alemania 2006,Zinedine Zidane,Francia,Fabio Cannavaro,Italia,Andrea Pirlo,Italia
7,Sudáfrica 2010,Diego Forlán,Uruguay,Wesley Sneijder,Países Bajos,David Villa,España
8,Brasil 2014,Lionel Messi,Argentina,Thomas Müller,Alemania,Arjen Robben,Países Bajos
9,Rusia 2018,Luka Modrić,Croacia,Eden Hazard,Bélgica,Antoine Griezmann,Francia


## 4 - Conexion con una **API**

Veamos un muy breve ejemplo de interacción con una API



In [78]:
# Cargamos el módulo requests
import requests

In [95]:
# Vamos a consultar la api de Github de pandas
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
# Realizamos una operacion GET a la url
r = requests.get(url)

In [100]:
r.json()[0:2]

[{'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'CONTRIBUTOR',
  'body': 'Add Reversed so that it fulfills all properties of Sequence\r\n\r\n- [ ] closes #xxxx (Replace xxxx with the Github issue number)\r\n- [ ] [Tests added and passed](https://pandas.pydata.org/pandas-docs/dev/development/contributing_codebase.html#writing-tests) if fixing a bug or adding a new feature\r\n- [ ] All [code checks passed](https://pandas.pydata.org/pandas-docs/dev/development/contributing_codebase.html#pre-commit).\r\n- [ ] Added [type annotations](https://pandas.pydata.org/pandas-docs/dev/development/contributing_codebase.html#type-hints) to new arguments/methods/functions.\r\n- [ ] Added an entry in the latest `doc/source/whatsnew/vX.X.X.rst` file if fixing a bug or adding a new feature.\r\n',
  'closed_at': None,
  'comments': 1,
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/47669/comments',
  'created_at': '2022-07-11T18:13:14Z',

In [99]:
# Podemos construir un dataframe con esta respuesta
pd.DataFrame(r.json())

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,closed_at,author_association,active_lock_reason,draft,pull_request,body,reactions,timeline_url,performed_via_github_app,state_reason
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/47669,1301053153,PR_kwDOAA0YD847NjCK,47669,ENH: Add reversed to Series,...,None,CONTRIBUTOR,None,True,{'url': 'https://api.github.com/repos/pandas-d...,Add Reversed so that it fulfills all propertie...,{'url': 'https://api.github.com/repos/pandas-d...,https://api.github.com/repos/pandas-dev/pandas...,None,None
1,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/47668,1300793242,PR_kwDOAA0YD847MrBD,47668,ENH: TDA.__mul__ support non-nano,...,None,MEMBER,None,False,{'url': 'https://api.github.com/repos/pandas-d...,- [ ] closes #xxxx (Replace xxxx with the Gith...,{'url': 'https://api.github.com/repos/pandas-d...,https://api.github.com/repos/pandas-dev/pandas...,None,None
2,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/47667,1300747628,I_kwDOAA0YD85Nh9Vs,47667,BUG: caching in CachedAccessor is problematic ...,...,None,CONTRIBUTOR,None,NaN,NaN,### Pandas version checks\r\n\r\n- [X] I have ...,{'url': 'https://api.github.com/repos/pandas-d...,https://api.github.com/repos/pandas-dev/pandas...,None,None
3,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/47666,1300440361,I_kwDOAA0YD85NgyUp,47666,ENH: rolling mean with limited number of obser...,...,None,NONE,None,NaN,NaN,#### Is your feature request related to a prob...,{'url': 'https://api.github.com/repos/pandas-d...,https://api.github.com/repos/pandas-dev/pandas...,None,None
4,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/47663,1300066443,I_kwDOAA0YD85NfXCL,47663,Why does `index.fillna()` have different argum...,...,None,CONTRIBUTOR,None,NaN,NaN,"In particular, why can't `index.fillna()` have...",{'url': 'https://api.github.com/repos/pandas-d...,https://api.github.com/repos/pandas-dev/pandas...,None,None
5,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/47659,1299811423,I_kwDOAA0YD85NeYxf,47659,BUG: read_json broken for S3 URL with non-null...,...,None,NONE,None,NaN,NaN,### Pandas version checks\r\n\r\n- [X] I have ...,{'url': 'https://api.github.com/repos/pandas-d...,https://api.github.com/repos/pandas-dev/pandas...,None,None
6,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github